In [ ]:
import rasterio
from rasterio import  features
from PIL import Image
from pathlib import Path
import numpy as np
from shapely import geometry
import datetime
import json

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def mask_to_poly(mask, tolerance):
    mask = np.array(mask) * 255
    mask = features.sieve(mask, 25)
    pp = []
    for r,v in features.shapes(mask, mask>0):
        cds = r['coordinates']
        #print(v, len(cds))
        if len(cds) > 1:
            raise Exception
        if v>0:
            poly = np.array(cds)[0]
            shp = geometry.Polygon(poly)
            poly_s = shp.simplify(tolerance=tolerance)
            if shp.area > 1:
                poly = np.array(poly_s.exterior.xy).T
                pp.append(poly)
    return pp

def create_ann(src, dst, polys):                                                         
    fixed_polys = []                                                                 
    for i, poly in enumerate(polys):               
        if isinstance(poly, np.ndarray):
            poly = poly.tolist()
        poly = dict(geometry=dict(coordinates=[poly]))                     
        fixed_polys.append(poly)

    timestamp = '{:%Y_%b_%d_%H_%M_%S}'.format(datetime.datetime.now())               
    fn = dst / (src.stem + "|" + timestamp + '.json')
    print(f'Saving annotations at {fn}')                                       

    with open(str(fn), 'w') as f:                                              
        json.dump(fixed_polys, f, indent=4)

In [ ]:
src = Path('../input/preprocessed/rle_1024/masks/10044.png')
dst = src.parent.parent / 'polys'
dst.mkdir(exist_ok=True)

mask = Image.open(str(src))
Image.fromarray(np.array(mask)*255).resize((128,128))

In [ ]:
polys = mask_to_poly(mask, 2)

In [ ]:
p = polys[0]

In [ ]:
[len(p) for p in polys]

In [ ]:
create_ann(src, dst, polys)

In [ ]:
plt.imshow(mask > 0)

In [ ]:
uni = None
for p in pp:
    
    if uni is not None:
        uni = uni.union(p)
    else:
        uni = p

In [ ]:
uni